# 准备工作

## 前序数据预处理

In [2]:
#!/usr/bin/env python
# coding: utf-8

# 批量处理文件夹中所有文件
import os

# 获取folder_path下所有文件路径，储存在file_paths里
file_paths = []
folder_path = '../data_base/knowledge_path/VMAX-S'
for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        file_paths.append(file_path)
print(file_paths)

from langchain.document_loaders.pdf import PyMuPDFLoader
# from langchain.document_loaders.markdown import UnstructuredMarkdownLoader

# 遍历文件路径并把实例化的loader存放在loaders里
loaders = []

for file_path in file_paths:
    file_type = file_path.split('.')[-1]
    if file_type == 'pdf':
        loaders.append(PyMuPDFLoader(file_path))
    else:
        print(f"Unsupported file type: {file_type} for file {file_path}")

# 下载文件并存储到text
# 加载所有文档内容到 texts
texts = []
for loader in loaders:
    texts.extend(loader.load())  # 关键步骤：初始化 texts

    
# 作数据清洗
# 修改后的数据清洗部分（替换原始代码中对应段落）
import re

# 预编译正则表达式（提升效率）
linebreak_pattern = re.compile(
    r'(?<![\\u4e00-\\u9fff])\n(?![\\u4e00-\\u9fff])',  # 负向断言匹配非中文环境换行
    flags=re.DOTALL
)
space_pattern = re.compile(r'[ 　]+')  # 匹配半角/全角空格
special_chars = ['•', '▪', '▫', '▶', '®', '©']  # 可扩展的干扰符号列表

# 替换原始代码中的清洗循环
for text in texts:
    # 1. 清理非中文环境换行
    text.page_content = re.sub(
        linebreak_pattern,
        lambda m: m.group().replace('\n', ''),
        text.page_content
    )

    # 2. 批量清理特殊符号
    for char in special_chars:
        text.page_content = text.page_content.replace(char, '')

    # 3. 安全删除空格（保留URL等特殊场景）
    text.page_content = space_pattern.sub('', text.page_content)




#导入文本分割器
from langchain_text_splitters import RecursiveCharacterTextSplitter
''' 
* RecursiveCharacterTextSplitter 递归字符文本分割
RecursiveCharacterTextSplitter 将按不同的字符递归地分割(按照这个优先级["\n\n", "\n", " ", ""])，
    这样就能尽量把所有和语义相关的内容尽可能长时间地保留在同一位置
RecursiveCharacterTextSplitter需要关注的是4个参数：

* separators - 分隔符字符串数组
* chunk_size - 每个文档的字符数量限制
* chunk_overlap - 两份文档重叠区域的长度
* length_function - 长度计算函数
'''


# 知识库中单段文本长度
CHUNK_SIZE = 512

# 知识库中相邻文本重合长度
OVERLAP_SIZE = 50


# 使用递归字符文本分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE
)


split_docs = text_splitter.split_documents(texts)
print(f"切分后的文件数量：{len(split_docs)}")



print(f"切分后的字符数（可以用来大致评估 token 数）：{sum([len(doc.page_content) for doc in split_docs])}")



split_docs[300].page_content



['../data_base/knowledge_path/VMAX-S\\ZXVMAX-S（V6.20.80.02）告警处理.pdf', '../data_base/knowledge_path/VMAX-S\\ZXVMAX-S（V6.20.80.02）故障管理概述.pdf', '../data_base/knowledge_path/VMAX-S\\ZXVMAX-S（V6.23）产品描述（5GC业务）.pdf', '../data_base/knowledge_path/VMAX-S\\ZXVMAX-S（V6.23）产品描述（上网日志业务）.pdf', '../data_base/knowledge_path/VMAX-S\\ZXVMAX-S（V6.23）产品描述（数据业务）.pdf', '../data_base/knowledge_path/VMAX-S\\ZXVMAX-S（V6.23）产品描述（端到端业务）.pdf', '../data_base/knowledge_path/VMAX-S\\ZXVMAX-S（V6.23）产品描述（语音业务）.pdf']


E:\learn-llm-rag-easily\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


切分后的文件数量：891
切分后的字符数（可以用来大致评估 token 数）：319269


"ZXVMAX-S多维价值分析系统告警处理\n4.修改配置项'Agent的Java堆栈大小'的参数值，重启已修正的flume角色实例；\n5.等待5分钟，检查告警是否已恢复。如果恢复则结束，如果没恢复则进行第6步；\n6.以root用户登录产生该告警的服务器；\n7.执行ps-ef|grepFlumeAgent命令，获取flume服务（FlumeAgent进程）的PID；\n8.执行jmap-heap<PID>命令，检查flume服务（FlumeAgent进程）'Agent的Java堆栈大小'配置是否生效（蓝色字体的值）。如果不生效则跳到第10步，如果生效则进行第9步；\n9.等待5分钟，检查告警是否已恢复。如果恢复则结束，如果没恢复则进行第10步；\n10.备案，等待5分钟，执行步骤8，查看flume进程的堆内存配置是否生效。生效则结束，不生效则通知上级维护人员进一步排查。\n1.238124010036FlumeAgent进程5分钟意外退出次数告警描述"

## 前序embedding模型

In [16]:
# ZhipuAIEmbeddings模型
# import os
# from dotenv import load_dotenv, find_dotenv

# _ = load_dotenv(find_dotenv())

# from langchain_community.embeddings import ZhipuAIEmbeddings

# my_emb = ZhipuAIEmbeddings(
#     model="embedding-2",
#     api_key = os.environ['ZHIPUAI_API_KEY'],
# )

In [17]:
# from langchain_ollama.embeddings import OllamaEmbeddings

# # 初始化嵌入模型
# my_emb = OllamaEmbeddings(
#     base_url='http://localhost:11434',
#     model="bge-m3:latest"
# )

In [1]:
from langchain_community.embeddings import XinferenceEmbeddings

# 使用API代理服务提高访问稳定性
my_emb = XinferenceEmbeddings(
    server_url="http://120.79.252.32:9997", 
    model_uid="my_qwen3_embed_0.6b"
)


# 构建Milvus向量库

In [ ]:
import os
from langchain_community.vectorstores import Milvus
from langchain_core.documents import Document


# 创建嵌入模型
my_emb = XinferenceEmbeddings(
    server_url="http://120.79.252.32:9997", 
    model_uid="my_qwen3_embed_0.6b"
)

# 向量库创建
# connection_args = {
#     "uri": "tcp://120.79.252.32:19530"
# }

connection_args = {
    "host": "120.79.252.32",
    "port": 19530,
}

# 定义每批处理的文档数量
batch_size = 30

# 如果只想导入部分数据
# split_docs = split_docs[:3]
try:
    # 计算总批次数
    total_batches = (len(split_docs) + batch_size - 1) // batch_size
    
    # 初始化向量数据库（如果是第一次创建）
    vectordb = None
    
    for batch_num in range(total_batches):
        # 计算当前批次的起始和结束索引
        start_idx = batch_num * batch_size
        end_idx = min((batch_num + 1) * batch_size, len(split_docs))
        
        # 获取当前批次的文档
        batch_docs = split_docs[start_idx:end_idx]
        
        print(f"正在处理第 {batch_num + 1}/{total_batches} 批文档 (文档 {start_idx}-{end_idx-1})")

        if batch_num == 0:
            # 第一次创建向量数据库
            vectordb = Milvus.from_documents(
            documents=batch_docs,
            embedding=my_emb,
            collection_name="ZXVMAXS1121",
            drop_old=False,
            connection_args=connection_args,
            )
            
            # 如果使用Milvus的混合检索
            # 创建 Milvus VectorStore，实现 dense + sparse 混合检索 
            # vectordb = Milvus.from_documents( 
            # documents=batch_docs, 
            # embedding=my_emb, # 用于语义检索的 dense 向量 
            # builtin_function=BM25BuiltInFunction(), # BM25 的 sparse 全文检索 
            # vector_field=["dense", "sparse"], # 指定两个向量字段名称 
            # connection_args=connection_args, 
            # collection_name="ZXVMAXS6",
            # consistency_level="Strong", 
            # drop_old=True, # 若已有旧 collection，可删除重建 
            # )

        else:
            # 后续批次添加到现有集合
            vectordb.add_documents(batch_docs)
        
        # 每批处理后持久化
        # vectordb.persist()
        print(f"第 {batch_num + 1} 批文档已成功导入并持久化")
    
    print("所有文档已成功导入并持久化到向量数据库。")
    
except Exception as e:
    print(f"处理过程中发生错误: {e}")

# 连接数据库进行测试

In [8]:
from langchain_community.vectorstores import Milvus

my_emb = XinferenceEmbeddings(
    server_url="http://120.79.252.32:9997", 
    model_uid="my_qwen3_embed_0.6b"
)

# connection_args = {
#     "uri": "tcp://120.79.252.32:19530"
# }

connection_args = {
    "host": "120.79.252.32",
    "port": 19530,
}
# Milvus 连接参数
vectordb = Milvus(
        embedding_function=my_emb,
        collection_name="ZXVMAXS",  # Milvus 集合名称
        connection_args=connection_args,
    )




C:\Users\will\AppData\Local\Temp\ipykernel_40932\81257760.py:17: LangChainDeprecationWarning: The class `Milvus` was deprecated in LangChain 0.2.0 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-milvus package and should be used instead. To use it run `pip install -U :class:`~langchain-milvus` and import as `from :class:`~langchain_milvus import MilvusVectorStore``.
  vectordb = Milvus(


In [13]:
from pymilvus import connections, utility

def list_all_collections():
    """获取所有collection列表"""
    try:
        # 连接到Milvus（使用与vectordb相同的参数）
        connections.connect(
            alias="default",
            host="120.79.252.32",
            port="19530"
        )
        
        # 获取所有collection
        collections = utility.list_collections()
        print("所有Collection列表:")
        for i, coll_name in enumerate(collections, 1):
            print(f"{i}. {coll_name}")
            
        return collections
        
    except Exception as e:
        print("获取collection列表失败:", e)
        return []
    finally:
        # 断开连接
        connections.disconnect("default")

# 执行
collections = list_all_collections()
print(f"总共找到 {len(collections)} 个collection")

所有Collection列表:
1. ZXVMAXS
2. ZXVMAXS1121
总共找到 2 个collection


In [11]:
# 测试相似性搜索
query = "ZXVMAXS的5G上网日志有那些功能？"
results = vectordb.similarity_search(query, k=2)
print("相似性搜索结果:")
for i, doc in enumerate(results):
    print(f"结果 {i+1}: {doc.page_content} - 元数据: {doc.metadata}")

相似性搜索结果:
结果 1: 1.106113030103thrift进程文件描述符使用百分比............................60
1.107113030106thrift进程每分钟垃圾回收时间所占的百分比..................61
1.108113030107thrift进程5分钟意外退出次数...............................61
1.109113040111thrift2进程文件描述符使用百分比...........................61
1.110113040114thrift2进程每分钟垃圾回收时间所占的百分比.................62
1.111113040115thrift2进程5分钟意外退出次数..............................62
1.112113040117RegionServer进程客户端请求时延10毫秒内次数占比............63
1.113113040118RegionServer进程客户端请求时延2000毫秒内次数占比..........63 - 元数据: {'pk': 461578465653297760, 'producer': 'Apache FOP Version 2.3', 'creator': 'DITA Open Toolkit', 'creationdate': '2022-06-23T16:34:22+08:00', 'source': '../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.20.80.02）告警处理.pdf', 'file_path': '../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.20.80.02）告警处理.pdf', 'total_pages': 330, 'format': 'PDF 1.4', 'title': '目录', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': "D:20220623163422+08'00'"

In [12]:
# 获取带相似度分数的结果
results_with_score = vectordb.similarity_search_with_score(query, k=3)
print("带分数的搜索结果:")
for doc, score in results_with_score:
    print(f"内容: {doc.page_content}, 分数: {score:.4f}")

带分数的搜索结果:
内容: 1.106113030103thrift进程文件描述符使用百分比............................60
1.107113030106thrift进程每分钟垃圾回收时间所占的百分比..................61
1.108113030107thrift进程5分钟意外退出次数...............................61
1.109113040111thrift2进程文件描述符使用百分比...........................61
1.110113040114thrift2进程每分钟垃圾回收时间所占的百分比.................62
1.111113040115thrift2进程5分钟意外退出次数..............................62
1.112113040117RegionServer进程客户端请求时延10毫秒内次数占比............63
1.113113040118RegionServer进程客户端请求时延2000毫秒内次数占比..........63, 分数: 6436.9302
内容: ZXVMAX-S多维价值分析系统告警处理可能原因确保实际填充率达到预期填充率处理步骤确保实际填充率达到预期填充率
7.2194000200303HTTP接口下行TCP重传报文数完整率(小时)告警描述确保实际填充率达到预期填充率告警级别警告可能原因确保实际填充率达到预期填充率处理步骤确保实际填充率达到预期填充率
7.2204000200304HTTP接口TCP建链响应时延（ms）完整率(小时)告警描述确保实际填充率达到预期填充率告警级别警告可能原因确保实际填充率达到预期填充率处理步骤确保实际填充率达到预期填充率
7.2214000200305HTTP接口TCP建链确认时延（ms）完整率(小时)告警描述确保实际填充率达到预期填充率
276
SJ-20220623151803-011|2022-06-20（R1.0）, 分数: 6496.2637
内容: ZXVMAX-S多维价值分析系统告警处理告警级别警告可能原因确保实际填充率达到预期填充率处理步骤确保实际填充率达到预期填充率
7.2634000200347HTTP接口窗口大小合规率(小时)告警